In [ ]:
# # ONLY IF REQUIRED, Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install navabilitysdk

In [ ]:
from navability.entities import *
from navability.services import *
from uuid import uuid4
import asyncio
import numpy as np

In [ ]:

client = NavAbilityHttpsClient()
context = Client("guest@navability.io", "SDKpy_" + str(uuid4())[0:4], "Tutorial4_" + str(uuid4())[0:4])



In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
MapVizApp(context)


In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

In [ ]:

result_id = await addVariable(client, context, Variable("c0", VariableType.Pose2.value))
result_id = await addVariable(client, context, Variable("c1", VariableType.Pose2.value))
result_id = await addVariable(client, context, Variable("c2", VariableType.Pose2.value))
result_id = await addVariable(client, context, Variable("c3", VariableType.Pose2.value))
result_id = await addVariable(client, context, Variable("door_prior", VariableType.Pose2.value))
result_id = await addVariable(client, context, Variable("door_dual", VariableType.Pose2.value))


prior_distr= np.power(np.diag([0.1, 0.1, 0.01]),2)
dual_distr= np.power(np.diag([0.5, 0.5, np.sqrt(np.pi)]),2)


f = Factor("l0f1", "PriorPose2", ["c0"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[0.,0, 0], cov=prior_distr)).dump()))
result_id = await addFactor(client, context, f) 
f = Factor("c1f1", "PriorPose2", ["c1"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20.,0, np.pi/2], cov=prior_distr)).dump()))
result_id = await addFactor(client, context, f) 
f = Factor("c2f1", "PriorPose2", ["c2"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20.,10, np.pi], cov=prior_distr)).dump()))
result_id = await addFactor(client, context, f) 
f = Factor("c3f1", "PriorPose2", ["c3"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[0.,10, -np.pi/2], cov=prior_distr)).dump()))
result_id = await addFactor(client, context, f) 

f = Factor("door_priorf1", "PriorPose2", ["door_prior"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20., 4, 0], cov=prior_distr)).dump()))
result_id = await addFactor(client, context, f) 
f = Factor("door_dualf1", "PriorPose2", ["door_dual"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20., 4, np.pi], cov=dual_distr)).dump()))
result_id = await addFactor(client, context, f) 



In [ ]:

result_id = await addVariable(client, context, Variable("x0", VariableType.Pose2.value))


f = Factor("x0c0c1c2c3f1", 
           "Pose2Pose2", 
           ["x0", "c0", "c1", "c2", "c3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[-2.,-2, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(client, context, f) 

await waitForCompletion(client, [result_id])

In [ ]:
solve_request = await solveSession(client, context)
print('running solve...')

await waitForCompletion(client, [solve_request], maxSeconds=180)

In [ ]:
result_id = await addVariable(client, context, Variable("x1", VariableType.Pose2.value))

f = Factor("x0x1f1", 
           "Pose2Pose2", 
           ["x0", "x1"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[4.,0, np.pi/2], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(),
           nullhypo=0.5)
)
result_id = await addFactor(client, context, f) 

f = Factor("x1c0c1c2c3f1", 
           "Pose2Pose2", 
           ["x1", "c0", "c1", "c2", "c3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[-2.,-4, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(client, context, f) 

await waitForCompletion(client, [result_id])


In [ ]:
solve_request = await solveSession(client, context)


In [ ]:
result_id = await addVariable(client, context, Variable("x2", VariableType.Pose2.value))

f = Factor("x1x2f1", 
           "Pose2Pose2", 
           ["x1", "x2"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[16.,0, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump())
)
result_id = await addFactor(client, context, f) 

f = Factor("x2c0c1c2c3f1", 
           "Pose2Pose2", 
           ["x2", "c0", "c1", "c2", "c3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[2.,-4, np.pi/2], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(client, context, f) 

await waitForCompletion(client, [result_id])


In [ ]:
solve_request = await solveSession(client, context)
print('running solve...')

await waitForCompletion(client, [solve_request], maxSeconds=180)

In [ ]:
f = Factor("x2doorf1", 
           "Pose2Pose2", 
           ["x2", "door_prior", "door_dual"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[2., 0, np.pi], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.5, 0.5])
)
result_id = await addFactor(client, context, f) 

await waitForCompletion(client, [result_id])

In [ ]:
solve_request = await solveSession(client, context)
print('running solve...')

await waitForCompletion(client, [solve_request], maxSeconds=180)